# RiskIQ PassiveTotal Python Library

#### *Illuminate Attack Surface Intelligence (ASI)*

## Getting Started

This notebook leverages the RiskIQ Illuminate / PassiveTotal API through the `passivetotal` Python library. 

Documentation for the library, including how to install it and configure API keys, are available here:
https://passivetotal.readthedocs.io/en/latest/getting-started.html

You will need API credentials to authenticate with the API server that provide access to the datasets queried in this notebook. Ask your RiskIQ contact for details or visit https://info.riskiq.net/ to contact the support team.

### Optional Dependencies

This notebook uses the `pandas` Python library primarily to improve the visual output of data tables retrieved from the API. You will need to install that library in your Python (virtual) environment (`pip install pandas`) or change the code examples to return a Python dictionary instead of a dataframe. Simply change `.as_df` to `.as_dict`.

Note that some examples may use special features in `pandas` to filter or aggregate data, but these can also be implemented in pure Python.

### Product Context

https://www.riskiq.com/solutions/attack-surface-intelligence/

### Setup Notebook
*If this returns errors, ensure you have followed the Getting Started document linked above to install necessary dependencies and configure your API keys.*

In [15]:
from passivetotal import analyzer
analyzer.init()

## Attack Surface Intelligence

### Your Attack Surface

Define a variable to store your organization's attack surface

In [ ]:
my_asi = analyzer.illuminate.AttackSurface.load()
my_asi

The `my_asi` variable here now stores an instance of `AttackSurface` object. To learn what you can do with this object, place your cursor after the variable name, add a dot (.), and press the (tab) key. You'll see a menu of options. 

The complete list of properties is available in the [reference docs](
https://passivetotal.readthedocs.io/en/latest/illuminate.html#passivetotal.analyzer.illuminate.AttackSurface).

---
RiskIQ assesses your Attack Surface by analyzing a set of insights and testing whether the discovered assets in your Attack Surface are impacted by each insight. These impacted assets are listed as observations, and are grouped into three levels: high, medium, and low.

To obtain the list of impacted assets, first enumerate the insights, either by a specific priority or across all priority levels. The most direct route is the `all_active_insights` property.

In [ ]:
my_asi.all_active_insights.as_dict

> This property is filtered to only the insights with observations, but the API provides all insights, even those without observations. To see them, use the `all_insights` property instead.

The `all_active_insights` property returns an object of type `AttackSurfaceInsights`. Complete details on the capability of this object are available [in the reference docs](https://passivetotal.readthedocs.io/en/latest/illuminate.html#passivetotal.analyzer.illuminate.AttackSurfaceInsights) and follow the same list of options available for most list-like Analyzer objects. 

To get started, loop through the `all_active_insights` property as if it was Python list. 

In [ ]:
for insight in my_asi.all_active_insights:
    print(insight)

The `all_active_insights` property returns an object of type `AttackSurfaceInsight` which can be printed like a string, but also offers additional properties. Use tab-completion here in Jupyter on one insight or consult [the docs](https://passivetotal.readthedocs.io/en/latest/illuminate.html#passivetotal.analyzer.illuminate.AttackSurfaceInsight).

For example, we can sort the high-priority insights by reverse order of observations, select the first insight in the list, and look at the observations for that insight.

In [ ]:
my_asi.high_priority_insights.sorted_by('observation_count', True)[0].observations

Observations are of type `AttackSurfaceObservations` which is also list-like in it's behavior. Complete details are in the [reference docs](https://passivetotal.readthedocs.io/en/latest/illuminate.html#passivetotal.analyzer.illuminate.AttackSurfaceObservations) but again, the easiest way to start is to simply iterate the list.

In [ ]:
for obs in my_asi.high_priority_insights.sorted_by('observation_count', True)[0].observations:
    print(obs)

Each observation is of type `AttackSurfaceObservation` and when printed simply shows the asset name, although many more details are available in [other properties](https://passivetotal.readthedocs.io/en/latest/illuminate.html#passivetotal.analyzer.illuminate.AttackSurfaceObservation) including the dates when the observation was last seen.

---
Consider using pandas DataFrames if you are working with ASI interactively in a notebook. Virtually every object offers an `as_df` property which is especially useful for lists.

In [ ]:
my_asi.high_priority_insights.as_df

In [ ]:
my_asi.high_priority_insights.only_active_insights[0].observations.as_df

> Notice the use of `only_active_insights` here to filter the list of insights to only those with observations. If you skip this step you may get an API error when you query for observations if none are available for that insight.

### Third-Party (Vendor) Attack Surfaces

Define a variable to store all third-party attack surfaces and load them from the API.

In [ ]:
vendor_asi = analyzer.illuminate.AttackSurfaces.load()
vendor_asi

> The list of third-party vendors is defined in your account settings in consultation with your RiskIQ account team. There are no options to change the composition of the list in the API.

The object returned is of type `AttackSurfaces` - this can be treated as a list, filtered, or displayed in several ways. Full details are in the [reference docs](
https://passivetotal.readthedocs.io/en/latest/illuminate.html#passivetotal.analyzer.illuminate.AttackSurfaces).

If you have a very large list of third-party vendors, the API will return the data one page at a time, but that will be handled automatically by the Python library.

This will return a list of third-party vendors (associated with Third-Party Intelligence module) and other third-party metadata (attack surface id, name of the vendor, if the name of the organization is your own, if the attack surface is a third-party vendor, number of active high priority, medium priority, and low priority assets linked to insight detected observations. 

In [ ]:
vendor_asi.as_df

---
There are several ways to filter this list to focus on a specific vendor, especially once you determine the asi_id that RiskIQ applies to it. Here, we use features from the `pandas` data library to filter the pandas DataFrame to include only those records with a name that matches a specific vendor. Note this search is case-sensistive.

In [ ]:
vendor_asi.as_df[vendor_asi.as_df['name'].str.contains('Rhythmic')]

Once we know the vendor's Attack Surfce ID we can load it by number.

In [12]:
rhythmic_asi = analyzer.illuminate.AttackSurface.load(553865)

This object behaves the same as the attack surface we retrieved for our own attack surface earlier in this notebook.

In [ ]:
rhythmic_asi.as_df.T

> The `T` property of pandas dataframes rotates the table 90 degrees which improves formatting when you only have one row of data.

We can return all active insights with the `all_active_insights` property.

In [ ]:
rhythmic_asi.all_active_insights.as_df

Insights can be treated like strings to make printing them easier, but remember there are more fields available on each insight.

In [ ]:
for insight in rhythmic_asi.all_active_insights:
    print(insight)

---
Using simple string matching, we can search a vendor's attack surface for a specific insight.

In [ ]:
for insight in rhythmic_asi.all_active_insights:
    if insight.name == 'ASI: REvil Ransomware Actors Exploit Kaseya VSA Software in Broad Supply Chain Attack':
        for obs in insight.observations:
            print (obs)

The `all_active_insights` property of an `AttackSurface` object offers a number of filtering options, including `filter_substring` that performs a case-insensitive match on any string field in the objects in that list. This is a property available on most `RecordList` type objects in the Analyzer.

In [ ]:
for insight in rhythmic_asi.all_active_insights.filter_substring(name='kaseya'):
    for obs in insight.observations:
        print(obs)

We can apply the same technique to search across all vendor attack surfaces. Here, we iterate (loop through) the `vendor_asi` variable we stored earlier that contains all third-party attack surfces, and then store the length of the insight list that matches our keyword. 

In [ ]:
for vendor in vendor_asi:
    kaseya_insights = len(vendor.all_active_insights.filter_substring(name='kaseya'))
    print(vendor.name, kaseya_insights) 